# Axisim Punching-Through Shear Test

This example couples two domains via an zero-thickness interface. 

In [8]:
%matplotlib widget
import time
from bmcs_expsim.utils.mlab_decorators import decorate_figure
from mayavi import mlab
import numpy as np
import warnings

In [9]:
from ibvpy.api import TStepBC
from ibvpy.bcond import BCSlice
from ibvpy.xmodel.xdomain_fe_grid_axisym import XDomainFEGridAxiSym
from ibvpy.xmodel.xdomain_interface import XDomainFEInterface

In [10]:
from ibvpy.fets import FETS2D4Q
from ibvpy.fets.fets1D5 import FETS1D52ULRH

In [11]:
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField

# Material models

In [12]:
from bmcs_matmod.slide.vslide_34 import Slide34
from ibvpy.tmodel.mats3D.mats3D_elastic.vmats3D_elastic import \
    MATS3DElastic

# FE discretization

In [13]:
n_x_e = 10
n_inner_y_e = 5
n_outer_y_e = 5
L_x = 200.0
R_in = 100.0
R_out = 200.0
xd_inner = XDomainFEGridAxiSym(coord_min=(0, 0),
                          coord_max=(L_x, R_in),
                          shape=(n_x_e, n_inner_y_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
xd_outer = XDomainFEGridAxiSym(coord_min=(0, R_in),
                          coord_max=(L_x, R_out),
                          shape=(n_x_e, n_outer_y_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
m1 = MATS3DElastic(E=28000, nu=0.3)
m2 = MATS3DElastic(E=28000, nu=0.3)

xd12 = XDomainFEInterface(
    I=xd_inner.mesh.I[1:-1, -1],
    J=xd_outer.mesh.I[1:-1, 0],
    fets=FETS1D52ULRH()
)
bond_m = Slide34(E_T=10000, E_N=10000)

fixed_y = BCSlice(slice=xd_inner.mesh[0, 0, 0, 0],
                 var='u', dims=[1], value=0)
fixed_outer_x = BCSlice(slice=xd_outer.mesh[0, :, 0, :],
                 var='u', dims=[0], value=0)
loaded_inner_x = BCSlice(slice=xd_inner.mesh[-1, :, -1, :],
                  var='u', dims=[0], value=-0.1)
bc1 = [fixed_y, fixed_outer_x, loaded_inner_x]

m = TStepBC(
    domains=[(xd_inner, m1),
             (xd_outer, m2),
             (xd12, bond_m),
             ],
    bc=bc1,  # + bc2,
)

m.hist.vis_record = {
    'strain': Vis3DTensorField(var='eps_ab'),
    #        'damage': Vis3DStateField(var='omega_a'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 0.1
s.tstep.fe_domain.serialized_subdomains

In [14]:
bond_m.interact()

In [7]:
xd12.hidden = True
s.run()

t:	 0.00

<lambdifygenerated-7>:4: RuntimeWarning: invalid value encountered in true_divide
  return (array([[I*select([greater(O, (abs(sqrt((-X_x + tau_pi_x/(1 - omega_T))**2 + (-X_y + tau_pi_y/(1 - omega_T))**2)) - (Z + bartau)*(sig_pi/(-omega_N*Dummy_448 + 1) + f_t**2*m/(Z + bartau - 2*f_t*m))*(Z + bartau - 2*f_t*m)/(f_t**2*m))*sign(f_t)*sign(m)),greater(O, (abs(sqrt((-X_x + tau_pi_x/(1 - omega_T))**2 + (-X_y + tau_pi_y/(1 - omega_T))**2)) + (Z + bartau + f_c0*m)*(sig_pi/(-omega_N*Dummy_448 + 1) + f_c0 - m*(-f_c**2 + 2*f_c*f_c0 - f_c0**2)/(Z + bartau + 2*f_c*m - f_c0*m))*(Z + bartau + 2*f_c*m - f_c0*m)/(m*(-f_c**2 + 2*f_c*f_c0 - f_c0**2)))*sign(m)*sign(-f_c + f_c0)),True], [(-X_x + tau_pi_x/(1 - omega_T))*(f_t**2*m**2 - 2*f_t*m*(Z + bartau) + (Z + bartau)**2)**2/((1 - omega_T)*(Z + bartau)*sqrt(((-X_x + tau_pi_x/(1 - omega_T))**2 + (-X_y + tau_pi_y/(1 - omega_T))**2)*(f_t**2*m**2 - 2*f_t*m*(Z + bartau) + (Z + bartau)**2)**2/((Z + bartau)*(-Z - bartau + f_t*m)**2*(Z + bartau - 2*f_t*m)) + (sig

(0), 
	 0.10(5), 
	 0.20(7), 
	 0.30(7), 
	 0.40(7), 
	 0.50(7), 
	 0.60(7), 
	 0.70(7), 
	 0.80(7), 
	 0.90(7), 
	 1.00(7), 



In [9]:
f_strain = mlab.figure()
strain_viz = Viz3DTensorField(vis3d=s.hist['strain'])
strain_viz.setup()
strain_viz.warp_vector.filter.scale_factor = 100.0
strain_viz.plot(s.tstep.t_n)
strain_viz.etc.filter
strain_viz.d.timestep=10

In [10]:
mlab.init_notebook(backend='ipy')
mlab.test_plot3d()

Notebook initialized with ipy backend.


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

In [11]:
#mlab.options.offscreen = True # 'envisage'
#f_strain = mlab.figure()
#scene = mlab.get_engine().scenes[-1]
#scene.name = 'strain'
strain_viz = Viz3DTensorField(vis3d=s.hist['strain'])
strain_viz.setup()
strain_viz.warp_vector.filter.scale_factor = 100.0
strain_viz.plot(s.tstep.t_n)
strain_viz.etc.filter


In [12]:
from tvtk.api import tvtk
surf_tvtk = mlab.pipeline.get_vtk_src(strain_viz.surface)[0]
surf_vtk = tvtk.to_vtk(surf_tvtk)
surf_vtk

(vtkmodules.vtkCommonDataModel.vtkPolyData)0x7f412c070b80

In [13]:
mlab.options.backend = 'envisage'
mlab.options.offscreen = False # 'envisage'
f_strain = mlab.figure()
scene = mlab.get_engine().scenes[-1]
scene.name = 'strain'
strain_viz = Viz3DTensorField(vis3d=s.hist['strain'])
strain_viz.setup()
strain_viz.warp_vector.filter.scale_factor = 100.0
strain_viz.plot(s.tstep.t_n)
strain_viz.etc.filter
decorate_figure(f_strain, strain_viz, 200, [70, 20, 0])

mlab.show()

In [ ]:
import k3d

model_matrix = (
     10.0,  0.0, 0.0, 0.0,
     0.0,  10.0, 0.0, 0.0,
     0.0,  0.0, 10.0, 0.0,
     0.0,  0.0, 0.0, 1.0
)
plot = k3d.plot()
cow3d = k3d.vtk_poly_data(surf_vtk)
#                         color_map=k3d.basic_color_maps.Jet, model_matrix=model_matrix)
plot += cow3d
plot.display()

In [ ]:

if False:
    f_damage = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'damage'
    damage_viz = Viz3DScalarField(vis3d=s.hist['damage'])
    damage_viz.setup()
    damage_viz.warp_vector.filter.scale_factor = 100.0
    damage_viz.plot(s.tstep.t_n)

#decorate_figure(f_damage, damage_viz)
decorate_figure(f_strain, strain_viz)
mlab.show()

In [ ]:
from mayavi.scripts import mayavi2

In [ ]:
@mayavi2.standalone
def view():
    from mayavi.sources.vtk_data_source import VTKDataSource
    from mayavi.modules.api import Outline, GridPlane

    mayavi.new_scene()
    src = VTKDataSource(data=sgrid)
    mayavi.add_source(src)
    mayavi.add_module(Outline())
    g = GridPlane()
    g.grid_plane.axis = 'x'
    mayavi.add_module(g)
    g = GridPlane()
    g.grid_plane.axis = 'y'
    mayavi.add_module(g)
    g = GridPlane()
    g.grid_plane.axis = 'z'
    mayavi.add_module(g)

view()
